# Подготовка данных для rfm моделей



In [1]:
%enable_full_walk
import sys
import numpy as np
import pandas as pd

import os
import functools as ft
root = ft.reduce(os.path.join, ['..'])

import pickle
import scipy
from scipy.sparse import coo_matrix

from sklearn.model_selection import train_test_split

def load_mapping(filename):
    map_path = ft.reduce(os.path.join, ['..','data','mapping' ,filename+'.pkl'])
    with open(map_path, 'rb') as handle:
        return pickle.load(handle)

In [2200]:
path = ft.reduce(os.path.join, ['..','data', 'raw', 'purchases.csv'])
df = pd.read_csv(path, dtype={'transaction_id': 'str',
                                'product_id': 'str',
                                'store_id': 'str',
                                'client_id': 'str'},
                 usecols=['client_id', 'transaction_id', 'transaction_datetime',\
                  'store_id', 'product_id', 'product_quantity', 'trn_sum_from_iss'])
                 
client_id_to_idx = load_mapping('client_id_to_idx')
product_id_to_idx = load_mapping('product_id_to_idx')
trans_id_to_idx = load_mapping('trans_id_to_idx')
store_id_to_idx = load_mapping('store_id_to_idx')
products_list = list(product_id_to_idx.values())

# Далее работаем с датасетом, где сохраняется только внутренняя индексация
df['transaction_idx'] = df.transaction_id.map(trans_id_to_idx)
df['product_idx'] = df.product_id.map(product_id_to_idx)
df['client_idx'] = df.client_id.map(client_id_to_idx)
df['store_idx'] = df.store_id.map(store_id_to_idx)

df.dropna(subset=['transaction_idx', 'product_idx', 'client_idx', 'store_idx'], inplace=True)

cols = ['transaction_datetime', 'transaction_idx', 'client_idx', 'store_idx', 'product_idx', 'product_quantity', 'trn_sum_from_iss']
df = df[cols]

# Сортируем по времени
df.sort_values('transaction_datetime', inplace=True)
df.reset_index(inplace=True, drop=True)

df[['transaction_idx', 'product_idx', 'client_idx', 'store_idx']] = df[['transaction_idx', 'product_idx', 'client_idx', 'store_idx']].astype(int)


In [2211]:
# Берем последнюю транзакцию каждого клиента
transactions = df[['transaction_idx', 'client_idx', 'transaction_datetime', 'store_idx']].drop_duplicates()
old_trans = transactions.groupby('client_idx').apply(lambda x: x[x.transaction_datetime == x.transaction_datetime.max()])
del transactions

# Расчет RFM агрегатов

In [2220]:
X_train = pd.read_csv(ft.reduce(os.path.join, [root, 'data', 'processed', 'client_hist_context_target_product.csv']), index_col=0)
# Выбираем только клиентов с историей
X_train = X_train[~X_train['is_new_client']]

old_trans_ = old_trans[old_trans.client_idx.isin(X_train.index.unique().values)].copy()

df['transaction_datetime'] = pd.to_datetime(df.transaction_datetime)
old_trans_['transaction_datetime'] = pd.to_datetime(old_trans_.transaction_datetime)

In [2222]:
h_columns = ['last_purch_days_' + str(x) for x in range(len(products_list))]
time_last_purch = np.full(len(products_list), np.inf)

# Количество дней после покупки товара
def get_last_purch_days(x):

    hist = df[(df.client_idx == x.client_idx) & (df.transaction_datetime < x.transaction_datetime)]

    hist = hist[['product_idx', 'transaction_datetime']]
    hist.drop_duplicates(keep='last', inplace=True)
    hist['cur_tdt'] = x.transaction_datetime
    hist.set_index('product_idx', inplace=True)
    hist['time_last_purch'] = (hist['cur_tdt'] - hist['transaction_datetime']).apply(lambda x: x.days)
    time_last_purch_ = time_last_purch.copy()
    time_last_purch_[hist.index.values] = hist['time_last_purch'].values
   
    return pd.Series(time_last_purch_, index=h_columns)

old_trans_last_purch = old_trans_.apply(get_last_purch_days, axis=1)

In [2223]:
h_columns = ['h_' + str(x) for x in range(len(products_list))]
zeros_hist = np.zeros(len(products_list)).astype(int)

# Количество покупок товара в истории
def get_history_products(x):

    hist = df[(df.client_idx == x.client_idx) & (df.transaction_datetime < x.transaction_datetime)]

    vc = hist.product_idx.value_counts()

    exp_hist = zeros_hist.copy()
    exp_hist[vc.index.values] = vc.values

    return pd.Series(exp_hist, index=h_columns)

old_trans_frequency = old_trans_.apply(get_history_products, axis=1)
old_trans_frequency.columns = ['hist_prod_freq_' + str(x) for x in range(len(products_list))]
old_trans_frequency = old_trans_frequency.div(old_trans_frequency.sum(axis=1), axis=0)

In [2224]:
h_columns = ['hist_purch_sum_' + str(x) for x in range(len(products_list))]
zeros_sum = np.zeros(len(products_list))

#Относительные траты на товар
def get_purch_sum(x):

    hist = df[(df.client_idx == x.client_idx) & (df.transaction_datetime < x.transaction_datetime)]

    hist = hist[['product_idx', 'trn_sum_from_iss']]
    hist = hist.groupby('product_idx').sum()
    
    zeros_sum_ = zeros_sum.copy()
    zeros_sum_[hist.index.values] = hist['trn_sum_from_iss'].values
   
    return pd.Series(zeros_sum_, index=h_columns)

old_trans_purch_summ = old_trans_.apply(get_purch_sum, axis=1)
old_trans_purch_relative_summ = old_trans_purch_summ.div(old_trans_purch_summ.sum(axis=1), axis=0)

In [2225]:
h_columns = ['hist_purch_period_sum_' + str(x) for x in range(len(products_list))]
zeros_sum = np.zeros(len(products_list))

# Сумма трат на товар за год
def get_purch_period_sum(x):

    hist = df[(df.client_idx == x.client_idx) & (df.transaction_datetime < x.transaction_datetime)].copy()
    
    hist['diff_days'] = hist.apply(lambda y: (x.transaction_datetime - y.transaction_datetime).days, axis=1)
    hist = hist[hist['diff_days'] < 366]

    hist = hist[['product_idx', 'trn_sum_from_iss']]
    hist = hist.groupby('product_idx').sum()
    
    zeros_sum_ = zeros_sum.copy()
    zeros_sum_[hist.index.values] = hist['trn_sum_from_iss'].values
   
    return pd.Series(zeros_sum_, index=h_columns)

old_trans_purch_period_summ = old_trans_.apply(get_purch_period_sum, axis=1)

In [2233]:
rfm = pd.concat([old_trans_purch_period_summ, old_trans_last_purch, old_trans_frequency, old_trans_purch_relative_summ], axis=1)
rfm = rfm.droplevel(1)
X_train_ = X_train.join(rfm, how='left')
X_train_.to_csv(ft.reduce(os.path.join, ['..','data', 'processed', 'rfm_client_hist_context_target_product.csv']), index=True)